## An Example of Importance Sampling for a Game

Consider a game where $X_1, X_2 \overset{\textrm{IID}}{\sim} \mathcal{U}[0,1]$ are drawn with a payoff of 

$$ 
Y = \text{payoff}(X_1,X_2) = \begin{cases} \$100, & 1.7 \le X_1 + X_2 \le 2, \\
 0, & 0 \le X_1 + X_2 < 1.7,
\end{cases}
$$

What is the expected payoff of this game?

### Vanilla Monte Carlo
With ordinary Monte Carlo we do the following:

$$ \mu = \mathbb{E}(Y) = \int_{[0,1]^2} \text{payoff}(x_1,x_2) \, \mathrm{d} x_1 \mathrm{d}x_2.$$

In [1]:
# Import necessary packages
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import time

/Users/yding/opt/anaconda3/envs/qmcpy/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


#### meanMC_CLT function

In [2]:
def meanMC_CLT(inputRandomFuc,absTol,relTol,alpha): #Simple version of meanMC_CLT
    nsig = 1000 # initial size of the sample
    inflate = 1.2 # inflation rate
    YY = inputRandomFuc(nsig)
    sigma  = np.std(YY,ddof = 1)
    hum = np.mean(YY)
    sigmaUpBound = sigma * inflate #upper ound on the standard deviation
    nmu = max(1, np.power(np.ceil(stats.norm.ppf(1-alpha/2)*sigmaUpBound/max(absTol,relTol)),2).astype(int) )  # number of samples needed for the error tolerance
    mu = np.mean(inputRandomFuc(nmu))
    nSample = nsig + nmu
    return mu, nSample

In [3]:
Y = lambda n: 100*(np.sum(np.random.rand(n,2),1)>=1.7) 
absTol = 0.005
start = time.time()
expPay, nSamples = meanMC_CLT(Y,absTol,0,0.01)
end = time.time()
print("The expectd payoff = ", f"{expPay:3.2f}", "+/-", absTol )
print("using ",f"{int(nSamples):,}","smaples and ", end-start, "seconds.")

The expectd payoff =  4.50 +/- 0.005
using  129,209,689 smaples and  7.209635019302368 seconds.


### Monte Carlo with Importance Sampling

We may add the importance sampling to increase the number of samples with
positive payoffs. Let 

$$ \boldsymbol{Z} = (X_1^{1/(p+1)}, X_2^{1/(p+1)}), \qquad
 \boldsymbol{X} \sim \mathcal{U}[0,1]^2. $$

This means that $Z_1$ and $Z_2$ are IID with common CDF $F(z) =
 z^{p+1}$ and common PDF $\varrho(z) = (p+1)z^{p}$.  Thus,

$$ \mu = \mathbb{E}(Y) = \int_{[0,1]^2}
 \frac{\text{payoff}(z_1,z_2)}{(p+1)^2(z_1z_2)^{p}} \, \varrho(z_1)
 \varrho(z_2) \, \mathrm{d} z_1 \mathrm{d}z_2 = \int_{[0,1]^2}
 \frac{\text{payoff}(x_1^{1/(p+1)},x_2^{1/(p+1)})}{(p+1)^2(x_1x_2)^{p/(p+1)}}
 \, \mathrm{d} x_1 \mathrm{d}x_2$$

In [4]:
p = 1
YIS = lambda x: (100/(p+1)**2)*(np.sum(x**(1/(p+1)),1)>=1.7)/((np.prod(x,1)**(p/(p+1))))
start = time.time()
expPay, nSamples = meanMC_CLT(lambda n: YIS(np.random.rand(n,2)),absTol,0,0.01)
end = time.time()
print("The expectd payoff = ", f"{expPay:3.2f}", "+/-", absTol )
print("using ",f"{int(nSamples):,}","smaples and ", end-start, "seconds.")

The expectd payoff =  4.50 +/- 0.005
using  45,092,225 smaples and  2.809810161590576 seconds.
